In [ ]:
import pandas as pd
import numpy as np

import ADPLearn as adp

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')

plt.rcParams['figure.figsize'] = (7, 15)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False

%matplotlib inline

In [381]:
department.gender = department.gender.astype(object)

In [416]:
def eda_chart(df, kind, **krargs):
    """_summary_
        kind
    Args:
        df (_type_): _description_
        kind (str): None
            - corr, pair
            - c1_n1_box, c1_n1_strip, c1_n1_violin
            - c1_n1_h1_violin
    """
    c, n, hue = None, None, None
    for k, v in krargs.items():
        if k=='c':
            c = krargs['c']
        elif k=='n':
            n = krargs['n']
        elif k=='hue':
            hue = krargs['hue']
    
    ## 상관관계 그래프
    if kind == 'corr':
        sns.heatmap(df.corr(), annot=True, vmax = 1,vmin = -1, 
                    cmap='coolwarm', linewidths=.5, fmt='.2f').set_title('Correlation')
    elif kind == 'na_heat':
        sns.heatmap(df[df.columns[df.dtypes != 'object']].T, 
                    cmap='Greens', linewidths=.5,).set_title('num_range')
    elif kind == 'pair':
        g = sns.pairplot(df, hue=hue, corner=True)
        g.fig.subplots_adjust(top=0.95)
        g.fig.suptitle('Pair')
    elif kind == 'c1_n1_box':
        g =sns.catplot(x=n, y=c, data=df, kind='box')
        g.fig.subplots_adjust(top=0.95)
        g.fig.suptitle(c)
    elif kind == 'c1_n1_strip':
        g =sns.catplot(x=n, y=c, data=df, kind='strip', alpha=.7, s=3)
        g.fig.subplots_adjust(top=0.95)
        g.fig.suptitle(c)
    elif kind == 'c1_n1_violin':
        g =sns.catplot(x=n, y=c, data=df, kind='violin', palette="coolwarm")
        g.fig.subplots_adjust(top=0.95)
        g.fig.suptitle(c)
    elif kind == 'c1_n1_h1_violin':
        g =sns.catplot(x=c, y=n, data=df, hue=hue, kind='violin', alpha=.7, split=True)
        g.fig.subplots_adjust(top=0.95)
        g.fig.suptitle(c)
    plt.show()

## 3). 변수축소 - 주성분 & 요인분석

| 주성분분석 | 요인분석 |
| --- | --- |
| 관찰기술(요인분석 유형) | 모델링 기술 |
| 최대 분산양 설명 | 데이터의 공분산을 설명 |
| 관측된 변수의 선형 조합 | 미관측 변수/요인 선형조합 |
| 구성요소 해석 불가 | 레이블링 및 해석 가능 |
| 차원 축소법 | 잠재변수 방법 |

3. 주성분분석 실행

In [ ]:
ncom, cev, loadings, pca_df = adp.pca(scaled_df, 0.85)

4. 주성분분석 결과 확인

In [ ]:
print('최적 요인 수: {}\n누적 성분 분산량: {:.3f}'.format(ncom, cev))

In [ ]:
loadings

In [ ]:
pca_df.head(5)

### (2) 요인분석

1. ADPLearn 모듈과 데이터를 로드

In [ ]:
swiss, _ = adp.loaddata('swiss')
swiss.head(3)

2. 데이터 표준정규화

In [ ]:
# 표준 정규화
scaled_df = adp.scaler(swiss, 'standard')
scaled_df.head(3)

3. 요인분석 적정성 검사
    - bartlett: 상관행렬이 항등행렬(변수간 관련이 없음)이라는 귀무가설 검정
    - kmo : 변수간 편상관(인자가 서로를 설명) 강도 조사, 1에 가까울 수록 유리

In [ ]:
test = ['bartlett', 'kmo']

for t in test:
    t_r, tr_pass = adp.test_fa(scaled_df, t)
    print('{} test: {}\n{}'.format(t, 'Pass' if tr_pass else 'Fail', t_r))

4. 요인분석 실행

In [ ]:
ncom, cev, loadings, fa_df = adp.factor_analysis(scaled_df, 0.85)

4. 요인분석 결과 확인
    - Cumulative Eigen Value : 전체 변수에 대한 설명력

In [ ]:
print('최적 요인 수: {}\n누적 아이젠 값: {:.3f}'.format(ncom, cev))

In [ ]:
loadings

In [ ]:
fa_df.head(5)

5. 요인분석 결과 검증
    - 크론바흐 알파: 주어진 변수들이 한개 그룹에 속할 수 있는지 내적일관성 측정, 0.7 이상이면 우수

In [ ]:
t_r = adp.cronbach_alpha(scaled_df, loadings, 0.3)